# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_id_text = []
    target_id_text = []
    for sentence in source_text.split("\n"):
        out = [source_vocab_to_int[w] for w in sentence.split()]
        source_id_text.append(out)
    for sentence in target_text.split("\n"):
        out = [target_vocab_to_int[w] for w in sentence.split()]
        out.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(out)
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [38]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, shape=(None,None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None,None), name='target')
    learning_rate = tf.placeholder(tf.float32, shape=(None), name='lr')
    keep_probability = tf.placeholder(tf.float32, shape=(None), name='keep_prob')
    
    
    target_sequence_length = tf.placeholder(tf.int32, shape=(None, ), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, shape=(None, ), name='source_sequence_length')

    return input, targets, learning_rate, keep_probability, target_sequence_length,\
           max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [39]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    target_data_slice = tf.strided_slice(
        target_data, [0, 0], [batch_size, -1], [1, 1])
    target_data = tf.concat([tf.fill(
        [batch_size, 1], target_vocab_to_int['<GO>']), target_data_slice], 1)
    return target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [40]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_embed = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    def create_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
        return drop

    enc_cell = tf.contrib.rnn.MultiRNNCell([create_cell(rnn_size) for _ in range(num_layers)])
    encoding_output, encoding_state = tf.nn.dynamic_rnn(enc_cell, enc_embed,
                                                        sequence_length=source_sequence_length,dtype=tf.float32)

    return encoding_output, encoding_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [41]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)

    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                impute_finished=True,
                                                                maximum_iterations=max_summary_length)[0]
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [42]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)

    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                 impute_finished=True,
                                                                 maximum_iterations=max_target_sequence_length)[0]
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [43]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def create_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                            initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
        return drop

    dec_cell = tf.contrib.rnn.MultiRNNCell([create_cell(rnn_size) for _ in range(num_layers)])

    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

    with tf.variable_scope("decode"):
        train_decoder_out = decoding_layer_train(encoder_state, dec_cell, dec_embed_input,
                                                 target_sequence_length, max_target_sequence_length, output_layer, keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_decoder_out = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings,
                                                 target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], max_target_sequence_length,
                                                 target_vocab_size, output_layer, batch_size, keep_prob)

    return (train_decoder_out, infer_decoder_out)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [44]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,
                                  source_sequence_length, source_vocab_size,
                                  enc_embedding_size)

    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)

    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state,
                                                                       target_sequence_length, max_target_sentence_length,
                                                                       rnn_size,
                                                                       num_layers, target_vocab_to_int, target_vocab_size,
                                                                       batch_size, keep_prob, dec_embedding_size)

    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [45]:
# Number of Epochs
epochs = 20
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 40
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 25
decoding_embedding_size = 25
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.8
display_step = 5

### Build the Graph
Build the graph using the neural network you implemented.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/local/lib/python3.5/dist-packages/tornado/ioloop.p

Batch and pad the source and target sequences

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/538 - Train Accuracy: 0.2626, Validation Accuracy: 0.3157, Loss: 4.9561
Epoch   0 Batch   10/538 - Train Accuracy: 0.2209, Validation Accuracy: 0.3157, Loss: 4.1155
Epoch   0 Batch   15/538 - Train Accuracy: 0.3158, Validation Accuracy: 0.3452, Loss: 3.5425
Epoch   0 Batch   20/538 - Train Accuracy: 0.2991, Validation Accuracy: 0.3469, Loss: 3.4293
Epoch   0 Batch   25/538 - Train Accuracy: 0.2992, Validation Accuracy: 0.3654, Loss: 3.3990
Epoch   0 Batch   30/538 - Train Accuracy: 0.2746, Validation Accuracy: 0.3468, Loss: 3.3438
Epoch   0 Batch   35/538 - Train Accuracy: 0.2807, Validation Accuracy: 0.3627, Loss: 3.2567
Epoch   0 Batch   40/538 - Train Accuracy: 0.3674, Validation Accuracy: 0.3734, Loss: 2.9491
Epoch   0 Batch   45/538 - Train Accuracy: 0.3465, Validation Accuracy: 0.3759, Loss: 3.0321
Epoch   0 Batch   50/538 - Train Accuracy: 0.3148, Validation Accuracy: 0.3757, Loss: 3.0897
Epoch   0 Batch   55/538 - Train Accuracy: 0.2820, Validation Accuracy

Epoch   0 Batch  450/538 - Train Accuracy: 0.4874, Validation Accuracy: 0.4822, Loss: 1.1055
Epoch   0 Batch  455/538 - Train Accuracy: 0.4553, Validation Accuracy: 0.4773, Loss: 1.0281
Epoch   0 Batch  460/538 - Train Accuracy: 0.4498, Validation Accuracy: 0.4831, Loss: 1.0666
Epoch   0 Batch  465/538 - Train Accuracy: 0.4418, Validation Accuracy: 0.4888, Loss: 1.0941
Epoch   0 Batch  470/538 - Train Accuracy: 0.4723, Validation Accuracy: 0.4833, Loss: 1.0583
Epoch   0 Batch  475/538 - Train Accuracy: 0.4767, Validation Accuracy: 0.4801, Loss: 1.0318
Epoch   0 Batch  480/538 - Train Accuracy: 0.4669, Validation Accuracy: 0.4819, Loss: 1.0329
Epoch   0 Batch  485/538 - Train Accuracy: 0.4881, Validation Accuracy: 0.4801, Loss: 1.0413
Epoch   0 Batch  490/538 - Train Accuracy: 0.4777, Validation Accuracy: 0.4821, Loss: 1.0051
Epoch   0 Batch  495/538 - Train Accuracy: 0.4840, Validation Accuracy: 0.5043, Loss: 1.0621
Epoch   0 Batch  500/538 - Train Accuracy: 0.4980, Validation Accuracy

Epoch   1 Batch  360/538 - Train Accuracy: 0.5400, Validation Accuracy: 0.5661, Loss: 0.7667
Epoch   1 Batch  365/538 - Train Accuracy: 0.5528, Validation Accuracy: 0.5707, Loss: 0.7323
Epoch   1 Batch  370/538 - Train Accuracy: 0.5301, Validation Accuracy: 0.5607, Loss: 0.7604
Epoch   1 Batch  375/538 - Train Accuracy: 0.5716, Validation Accuracy: 0.5762, Loss: 0.7064
Epoch   1 Batch  380/538 - Train Accuracy: 0.5633, Validation Accuracy: 0.5797, Loss: 0.7342
Epoch   1 Batch  385/538 - Train Accuracy: 0.5731, Validation Accuracy: 0.5790, Loss: 0.7234
Epoch   1 Batch  390/538 - Train Accuracy: 0.5956, Validation Accuracy: 0.5810, Loss: 0.7135
Epoch   1 Batch  395/538 - Train Accuracy: 0.5611, Validation Accuracy: 0.5751, Loss: 0.7492
Epoch   1 Batch  400/538 - Train Accuracy: 0.5655, Validation Accuracy: 0.5868, Loss: 0.7154
Epoch   1 Batch  405/538 - Train Accuracy: 0.5701, Validation Accuracy: 0.5785, Loss: 0.6985
Epoch   1 Batch  410/538 - Train Accuracy: 0.5604, Validation Accuracy

Epoch   2 Batch  270/538 - Train Accuracy: 0.6314, Validation Accuracy: 0.6523, Loss: 0.5841
Epoch   2 Batch  275/538 - Train Accuracy: 0.6168, Validation Accuracy: 0.6388, Loss: 0.5858
Epoch   2 Batch  280/538 - Train Accuracy: 0.6910, Validation Accuracy: 0.6493, Loss: 0.5415
Epoch   2 Batch  285/538 - Train Accuracy: 0.6728, Validation Accuracy: 0.6612, Loss: 0.5340
Epoch   2 Batch  290/538 - Train Accuracy: 0.6670, Validation Accuracy: 0.6451, Loss: 0.5533
Epoch   2 Batch  295/538 - Train Accuracy: 0.6928, Validation Accuracy: 0.6481, Loss: 0.5245
Epoch   2 Batch  300/538 - Train Accuracy: 0.6648, Validation Accuracy: 0.6465, Loss: 0.5449
Epoch   2 Batch  305/538 - Train Accuracy: 0.6916, Validation Accuracy: 0.6497, Loss: 0.5437
Epoch   2 Batch  310/538 - Train Accuracy: 0.6625, Validation Accuracy: 0.6475, Loss: 0.5685
Epoch   2 Batch  315/538 - Train Accuracy: 0.6594, Validation Accuracy: 0.6568, Loss: 0.5397
Epoch   2 Batch  320/538 - Train Accuracy: 0.6760, Validation Accuracy

Epoch   3 Batch  180/538 - Train Accuracy: 0.7623, Validation Accuracy: 0.7163, Loss: 0.4076
Epoch   3 Batch  185/538 - Train Accuracy: 0.7365, Validation Accuracy: 0.7283, Loss: 0.4085
Epoch   3 Batch  190/538 - Train Accuracy: 0.7046, Validation Accuracy: 0.7124, Loss: 0.4405
Epoch   3 Batch  195/538 - Train Accuracy: 0.7485, Validation Accuracy: 0.7138, Loss: 0.4205
Epoch   3 Batch  200/538 - Train Accuracy: 0.7369, Validation Accuracy: 0.7401, Loss: 0.4237
Epoch   3 Batch  205/538 - Train Accuracy: 0.7764, Validation Accuracy: 0.7152, Loss: 0.3894
Epoch   3 Batch  210/538 - Train Accuracy: 0.7011, Validation Accuracy: 0.7170, Loss: 0.3976
Epoch   3 Batch  215/538 - Train Accuracy: 0.7246, Validation Accuracy: 0.7156, Loss: 0.4368
Epoch   3 Batch  220/538 - Train Accuracy: 0.7091, Validation Accuracy: 0.7177, Loss: 0.4110
Epoch   3 Batch  225/538 - Train Accuracy: 0.7217, Validation Accuracy: 0.7061, Loss: 0.4204
Epoch   3 Batch  230/538 - Train Accuracy: 0.7002, Validation Accuracy

...

Epoch  16 Batch  345/538 - Train Accuracy: 0.9364, Validation Accuracy: 0.9252, Loss: 0.1034
Epoch  16 Batch  350/538 - Train Accuracy: 0.9271, Validation Accuracy: 0.9150, Loss: 0.1133
Epoch  16 Batch  355/538 - Train Accuracy: 0.9279, Validation Accuracy: 0.9237, Loss: 0.1052
Epoch  16 Batch  360/538 - Train Accuracy: 0.9037, Validation Accuracy: 0.9176, Loss: 0.0888
Epoch  16 Batch  365/538 - Train Accuracy: 0.9182, Validation Accuracy: 0.9189, Loss: 0.1049
Epoch  16 Batch  370/538 - Train Accuracy: 0.9266, Validation Accuracy: 0.9242, Loss: 0.0968
Epoch  16 Batch  375/538 - Train Accuracy: 0.9235, Validation Accuracy: 0.9135, Loss: 0.0932
Epoch  16 Batch  380/538 - Train Accuracy: 0.9203, Validation Accuracy: 0.9094, Loss: 0.0951
Epoch  16 Batch  385/538 - Train Accuracy: 0.9154, Validation Accuracy: 0.9087, Loss: 0.0989
Epoch  16 Batch  390/538 - Train Accuracy: 0.9250, Validation Accuracy: 0.9130, Loss: 0.0790
Epoch  16 Batch  395/538 - Train Accuracy: 0.9094, Validation Accuracy

Epoch  17 Batch  255/538 - Train Accuracy: 0.9303, Validation Accuracy: 0.9263, Loss: 0.0855
Epoch  17 Batch  260/538 - Train Accuracy: 0.8880, Validation Accuracy: 0.9213, Loss: 0.1017
Epoch  17 Batch  265/538 - Train Accuracy: 0.9023, Validation Accuracy: 0.9098, Loss: 0.1042
Epoch  17 Batch  270/538 - Train Accuracy: 0.9217, Validation Accuracy: 0.9160, Loss: 0.0928
Epoch  17 Batch  275/538 - Train Accuracy: 0.9180, Validation Accuracy: 0.9126, Loss: 0.1086
Epoch  17 Batch  280/538 - Train Accuracy: 0.9209, Validation Accuracy: 0.9151, Loss: 0.0805
Epoch  17 Batch  285/538 - Train Accuracy: 0.9288, Validation Accuracy: 0.9032, Loss: 0.0859
Epoch  17 Batch  290/538 - Train Accuracy: 0.9361, Validation Accuracy: 0.9107, Loss: 0.0872
Epoch  17 Batch  295/538 - Train Accuracy: 0.9421, Validation Accuracy: 0.9276, Loss: 0.0824
Epoch  17 Batch  300/538 - Train Accuracy: 0.9336, Validation Accuracy: 0.9228, Loss: 0.0912
Epoch  17 Batch  305/538 - Train Accuracy: 0.9301, Validation Accuracy

Epoch  18 Batch  165/538 - Train Accuracy: 0.9314, Validation Accuracy: 0.9160, Loss: 0.0858
Epoch  18 Batch  170/538 - Train Accuracy: 0.9096, Validation Accuracy: 0.9217, Loss: 0.0996
Epoch  18 Batch  175/538 - Train Accuracy: 0.9352, Validation Accuracy: 0.9213, Loss: 0.0887
Epoch  18 Batch  180/538 - Train Accuracy: 0.9358, Validation Accuracy: 0.9336, Loss: 0.0924
Epoch  18 Batch  185/538 - Train Accuracy: 0.9469, Validation Accuracy: 0.9155, Loss: 0.0878
Epoch  18 Batch  190/538 - Train Accuracy: 0.9260, Validation Accuracy: 0.9164, Loss: 0.1128
Epoch  18 Batch  195/538 - Train Accuracy: 0.9433, Validation Accuracy: 0.9180, Loss: 0.1108
Epoch  18 Batch  200/538 - Train Accuracy: 0.9307, Validation Accuracy: 0.9228, Loss: 0.0836
Epoch  18 Batch  205/538 - Train Accuracy: 0.9343, Validation Accuracy: 0.9206, Loss: 0.0833
Epoch  18 Batch  210/538 - Train Accuracy: 0.9064, Validation Accuracy: 0.9283, Loss: 0.1031
Epoch  18 Batch  215/538 - Train Accuracy: 0.9393, Validation Accuracy

Epoch  19 Batch   75/538 - Train Accuracy: 0.9174, Validation Accuracy: 0.9347, Loss: 0.1017
Epoch  19 Batch   80/538 - Train Accuracy: 0.9328, Validation Accuracy: 0.9302, Loss: 0.0939
Epoch  19 Batch   85/538 - Train Accuracy: 0.9354, Validation Accuracy: 0.9318, Loss: 0.0858
Epoch  19 Batch   90/538 - Train Accuracy: 0.9228, Validation Accuracy: 0.9206, Loss: 0.1057
Epoch  19 Batch   95/538 - Train Accuracy: 0.9066, Validation Accuracy: 0.9247, Loss: 0.0777
Epoch  19 Batch  100/538 - Train Accuracy: 0.9437, Validation Accuracy: 0.9350, Loss: 0.0805
Epoch  19 Batch  105/538 - Train Accuracy: 0.9459, Validation Accuracy: 0.9265, Loss: 0.0838
Epoch  19 Batch  110/538 - Train Accuracy: 0.9240, Validation Accuracy: 0.9325, Loss: 0.0874
Epoch  19 Batch  115/538 - Train Accuracy: 0.9480, Validation Accuracy: 0.9297, Loss: 0.0952
Epoch  19 Batch  120/538 - Train Accuracy: 0.9475, Validation Accuracy: 0.9364, Loss: 0.0791
Epoch  19 Batch  125/538 - Train Accuracy: 0.9345, Validation Accuracy

Epoch  19 Batch  520/538 - Train Accuracy: 0.9398, Validation Accuracy: 0.9226, Loss: 0.0916
Epoch  19 Batch  525/538 - Train Accuracy: 0.9427, Validation Accuracy: 0.9233, Loss: 0.0929
Epoch  19 Batch  530/538 - Train Accuracy: 0.9146, Validation Accuracy: 0.9274, Loss: 0.0998
Epoch  19 Batch  535/538 - Train Accuracy: 0.9302, Validation Accuracy: 0.9347, Loss: 0.0818
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [49]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [50]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [51]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

TypeError: object of type 'NoneType' has no len()

## Translate
This will translate `translate_sentence` from English to French.

In [52]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from checkpoints/dev


TypeError: object of type 'NoneType' has no len()

## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.